In [15]:
import skfuzzy.cluster as fuzz
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Importar o conjunto de dados
url1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data"
url2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.test"

cols = ["CLASS", "REGION-CENTROID-COL", "REGION-CENTROID-ROW", "REGION-PIXEL-COUNT", "SHORT-LINE-DENSITY-5", "SHORT-LINE-DENSITY-2", "VEDGE-MEAN", "VEDGE-SD", "HEDGE-MEAN", "HEDGE-SD", "INTENSITY-MEAN","RAWRED-MEAN","RAWBLUE-MEAN","RAWGREEN-MEAN","EXRED-MEAN","EXBLUE-MEAN","EXGREEN-MEAN","VALUE-MEAN","SATURATION-MEAN","HUE-MEAN"]

df1 = pd.read_csv(url1, header=2, names=cols)
df2 = pd.read_csv(url2, header=2, names=cols)
df3 = pd.concat([df1, df2]).drop(["REGION-CENTROID-COL", "REGION-CENTROID-ROW", "REGION-PIXEL-COUNT"], axis=1)
best_Us = []

In [2]:
df1 = df3.iloc[:,4:9] 
df2 = df3.iloc[:,10:19]
df3 = df3.iloc[:,4:19]

dfs = [df1,df2,df3]

In [3]:
X = df3.iloc[:, :-1].to_numpy(dtype=np.float32)
y = df3.iloc[:, -1].to_numpy()
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [4]:
#Para m=2
n_clusters = 7

for dfs in range(3): #Aqui selecionamos os 3 datasets
    best_obj_func = np.inf # Executando o algoritmo FCM com a distância city-block personalizada
    best_U = None # Melhor resultado
    for i in range(50):
        cntr, u, u0, d, jm, p, fpc = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
        if jm.mean() < best_obj_func:
            best_obj_func = jm.mean()
            best_U = u
    best_Us.append(best_U) #best_Us contém todas as melhores matrizes de partição fuzzy obtidas no algoritmo FCM

best_jm = np.inf

cntr, u, _, _, jm, _, _ = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=1000, metric='cityblock')
if jm.mean() < best_jm:
    best_jm = jm.mean()
    best_U = u
print("Melhor resultado de acordo com a Função objetivo:", best_jm)

Melhor resultado de acordo com a Função objetivo: 13259.407415797887


In [5]:
#Para m = 1.6
best_jm = np.inf
cntr, u, _, _, jm, _, _ = fuzz.cmeans(data=X.T, c=n_clusters, m=1.6, error=1e-3, maxiter=100, metric='cityblock')
if jm.mean() < best_jm:
    best_jm = jm.mean()
    best_U = u
print("Melhor resultado de acordo com a Função objetivo:", best_jm)

Melhor resultado de acordo com a Função objetivo: 22652.776315691364


In [6]:
#Para m = 1.1
best_jm = np.inf
cntr, u, _, _, jm, _, _ = fuzz.cmeans(data=X.T, c=n_clusters, m=1.1, error=1e-3, maxiter=100, metric='cityblock')
if jm.mean() < best_jm:
    best_jm = jm.mean()
    best_U = u
print("Melhor resultado de acordo com a Função objetivo:", best_jm)

Melhor resultado de acordo com a Função objetivo: 36214.64616619606


In [7]:
#MODIFIED PARTITION COEFFICIENT

#Para df1
cntr, u, u0, d, jm, p, fpc = fuzz.cmeans(data=df1.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
FE = -np.sum(u*np.log(u))
PE = -np.sum(u.mean(axis=1)*np.log(u.mean(axis=1)))
PC = (FE - PE) / np.log(n_clusters)
print(f"Modified Partition Coefficient (df1): {PC} Partition entropy (df1): {PE} ")

#Para df2
cntr, u, _, _, _, _, _ = fuzz.cmeans(data=df2.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
FE = -np.sum(u*np.log(u))
PE = -np.sum(u.mean(axis=1)*np.log(u.mean(axis=1)))
PC = (FE - PE) / np.log(n_clusters)
print(f"Modified Partition Coefficient (df2): {PC} Partition entropy (df2): {PE} ")

#Para df3
cntr, u, u0, d, jm, p, fpc = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
FE = -np.sum(u*np.log(u))
PE = -np.sum(u.mean(axis=1)*np.log(u.mean(axis=1)))
PC = (FE - PE) / np.log(n_clusters)
print(f"Modified Partition Coefficient (df3): {PC} Partition entropy (df3): {PE} ")


Modified Partition Coefficient (df1): 611.7195100397478 Partition entropy (df1): 1.4762017904762232 
Modified Partition Coefficient (df2): 859.0225233306347 Partition entropy (df2): 1.8994142429450755 
Modified Partition Coefficient (df3): 1092.5981212658444 Partition entropy (df3): 1.8724476749105976 


In [8]:
# Definir o número de clusters
n_clusters = 7

# Fuzzy c-means para cada dataset
cntr1, u1, _, _, _, _, _ = fuzz.cmeans(data=df1.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
cntr2, u2, _, _, _, _, _ = fuzz.cmeans(data=df2.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')
cntr3, u3, _, _, _, _, _ = fuzz.cmeans(data=X.T, c=n_clusters, m=2, error=1e-3, maxiter=100, metric='cityblock')

# Obter as partições nítidas
u1_crisp = np.argmax(u1, axis=0)
u2_crisp = np.argmax(u2, axis=0)
u3_crisp = np.argmax(u3, axis=0)

# Imprimir os resultados das partições CRISP
print("Partição nítida do conjunto de dados 1:")
print(u1_crisp)
print("Partição nítida do conjunto de dados 2:")
print(u2_crisp)
print("Partição nítida do conjunto de dados 3:")
print(u3_crisp)

Partição nítida do conjunto de dados 1:
[3 3 3 ... 3 3 3]
Partição nítida do conjunto de dados 2:
[1 1 1 ... 1 1 1]
Partição nítida do conjunto de dados 3:
[5 5 5 ... 5 5 3]


In [9]:
# ÍNDICE DE RAND AJUSTADO
# Converter partição fuzzy para crisp
from sklearn.metrics import adjusted_rand_score

labels1 = np.argmax(u1, axis=0)
labels2 = np.argmax(u2, axis=0)
labels3 = np.argmax(u3, axis=0)

y1 = df1.iloc[:, -1].to_numpy()
y2 = df2.iloc[:, -1].to_numpy()
y3 = df3.iloc[:, -1].to_numpy()


ari1 = adjusted_rand_score(y1, labels1)
ari2 = adjusted_rand_score(y2, labels2)
ari3 = adjusted_rand_score(y3, labels3)

# Print results
print("Adjusted Rand Index for dataset 1:", ari1)
print("Adjusted Rand Index for dataset 2:", ari2)
print("Adjusted Rand Index for dataset 3:", ari3)

Adjusted Rand Index for dataset 1: 0.02931265244977159
Adjusted Rand Index for dataset 2: 0.01437980247916568
Adjusted Rand Index for dataset 3: 0.014155398710015086


In [16]:
labels = np.argmax(u, axis=0)
true_labels = np.array([int(i)-1 for i in y]) # Convert y to 0-based labels
# Compute confusion matrix
cm = confusion_matrix(true_labels, labels)

In [17]:
def fmeasure(cm):
    l = len(cm)
    s = cm.sum()
    s_true = cm.sum(axis = 1)
    s_predict = cm.sum(axis = 0)
    f_score = np.zeros((l, l))
    for i in range(dfs):
        for j in range(7):
            f_score[i, j] = (2*cm[i, j])/(s_true[i]+s_predict[j])

    return ((f_score.max(axis =1)*s_true).sum())/s

fmeasure(cm)

0.22649005798115063

In [18]:
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import f1_score


# Generate 7 random partitions using KMeans
n_clusters = 7
partitions = []
for i in range(n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=i)
    labels = kmeans.fit_predict(X)
    partitions.append(labels)

# Compute ARI and F-measure for all pairs of partitions
for i in range(n_clusters):
    for j in range(i+1, n_clusters):
        ari = adjusted_rand_score(partitions[i], partitions[j])
        fmeasure = f1_score(partitions[i], partitions[j], average='weighted')
        print(f"Índice de Rand entre partições {i+1} e {j+1}: {ari:.3f}")
        print(f"F-measure entre partições {i+1} e {j+1}: {fmeasure:.3f}")


Índice de Rand entre partições 1 e 2: 0.995
F-measure entre partições 1 e 2: 0.233
Índice de Rand entre partições 1 e 3: 1.000
F-measure entre partições 1 e 3: 0.000
Índice de Rand entre partições 1 e 4: 1.000
F-measure entre partições 1 e 4: 0.000
Índice de Rand entre partições 1 e 5: 0.925
F-measure entre partições 1 e 5: 0.244
Índice de Rand entre partições 1 e 6: 0.995
F-measure entre partições 1 e 6: 0.002
Índice de Rand entre partições 1 e 7: 1.000
F-measure entre partições 1 e 7: 0.185
Índice de Rand entre partições 2 e 3: 0.995
F-measure entre partições 2 e 3: 0.322
Índice de Rand entre partições 2 e 4: 0.995
F-measure entre partições 2 e 4: 0.144
Índice de Rand entre partições 2 e 5: 0.929
F-measure entre partições 2 e 5: 0.438
Índice de Rand entre partições 2 e 6: 1.000
F-measure entre partições 2 e 6: 0.000
Índice de Rand entre partições 2 e 7: 0.995
F-measure entre partições 2 e 7: 0.000
Índice de Rand entre partições 3 e 4: 1.000
F-measure entre partições 3 e 4: 0.332
Índi